
### This script provides a dashboard for two NUTS 3 regions - EE001 (Estonia) and LT011 (Lithuania), for which shelter data has been obtained and calculated in preparation.


In [4]:
# Impport libraries
import pandas as pd
import json
import numpy as np
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import panel as pn
import holoviews as hv
import plotly.graph_objects as go
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from io import StringIO

# Initialize Panel and HoloViews extensions
pn.extension('plotly')
hv.extension('bokeh')

In [5]:
# Load region data from JSON file
# if it is not available, hardcoded data is used
try:
    with open("nuts_3_shelter_data.json", "r") as file:
        region_data = json.load(file)
    print(f"Successfully loaded data for {len(region_data)} regions.")
except (FileNotFoundError, json.JSONDecodeError) as e:
    # Fallback to hardcoded data if file doesn't exist or contains invalid JSON
    error_type = "not found" if isinstance(e, FileNotFoundError) else "contains invalid JSON"
    print(f"Warning: nuts_3_shelter_data.json {error_type}. Using default data.")

# Optional: Print the first region to verify data structure
if region_data:
    first_region_id = list(region_data.keys())[0]
    print(f"First region: {first_region_id} - {region_data[first_region_id]['name']}")

First region: EE001 - Põhja-Eesti


In [6]:
def generate_kpi_data(region_id):
    """
    Generate Key Performance Indicators (KPIs) for speedometer visualizations
    
    Parameters:
    region_id (str): NUTS-3 region identifier (e.g., "EE001")
    
    Returns:
    dict: Dictionary containing KPI values for shelters used and population coverage
    """
    region = region_data[region_id]
    
    # KPI 1: Calculate what percentage of potential shelters are actually being used
    # Example: If 250 official out of 1500 potential shelters, then 16.7%
    shelters_used = (region['official_shelters'][-1] / region['suggested_shelters'][-1]) * 100
    
    # KPI 2: Population Coverage - directly from the data
    # Represents percentage of population with access to shelter protection
    population_coverage = region['population_protected'][-1]
    
    return {
        # Cap values at 99% for display purposes (100% might cause visual issues on gauge)
        'shelters_used': min(shelters_used, 99),
        'population_coverage': min(population_coverage, 99)
    }

# Calculate KPI data for each region and add it to the region_data dictionary
# This pre-computes values we'll need later for visualizations
for region_id in region_data:
    region_data[region_id]['kpi_data'] = generate_kpi_data(region_id)

In [7]:
def load_nuts3_data():
    """
    Load and preprocess NUTS-3 GeoJSON data for Baltic countries.
    This function is a major performance optimization as it loads geographic data once
    at startup rather than repeatedly for each view change.
    
    Returns:
    tuple: (country_nuts3, selected_nuts3) containing preprocessed GeoJSON data
    """
    print("Loading NUTS-3 GeoJSON data...")
    
    # Load NUTS-3 GeoJSON data from Eurostat's server
    # This contains geographic boundaries for all European regions
    nuts_geojson_url = "https://gisco-services.ec.europa.eu/distribution/v2/nuts/geojson/NUTS_RG_01M_2021_4326_LEVL_3.geojson"
    
    # Load the GeoJSON data using GeoPandas
    # This creates a GeoDataFrame with geographic shapes and metadata
    nuts3 = gpd.read_file(nuts_geojson_url)
    
    # Filter for Baltic countries only to reduce data size
    baltic_countries = ['EE', 'LV', 'LT']  # Estonia, Latvia, Lithuania country codes
    baltic_nuts3 = nuts3[nuts3['CNTR_CODE'].isin(baltic_countries)].copy()
    
    # Further filter to only regions that we have detailed shelter data for
    available_regions = list(region_data.keys())
    selected_nuts3 = baltic_nuts3.loc[baltic_nuts3['NUTS_ID'].isin(available_regions)].copy()
    
    # Add the shelter data to the GeoDataFrame for mapping and visualization
    # This merges our custom data with the geographic data
    
    # Add latest official shelter count to each region
    selected_nuts3.loc[:, 'Official_Shelter_Latest'] = selected_nuts3['NUTS_ID'].apply(
        lambda x: region_data[x]['official_shelters'][-1] if x in region_data else np.nan
    )
    
    # Add latest suggested/potential shelter count to each region
    selected_nuts3.loc[:, 'Suggested_Shelter_Latest'] = selected_nuts3['NUTS_ID'].apply(
        lambda x: region_data[x]['suggested_shelters'][-1] if x in region_data else np.nan
    )
    
    # Add latest population protection percentage to each region
    selected_nuts3.loc[:, 'Population_protected'] = selected_nuts3['NUTS_ID'].apply(
        lambda x: region_data[x]['population_protected'][-1] if x in region_data else np.nan
    )
    
    # Add shelters_used percentage for coloring the map
    selected_nuts3.loc[:, 'Shelters_Used_Pct'] = selected_nuts3['NUTS_ID'].apply(
        lambda x: region_data[x]['kpi_data']['shelters_used'] if x in region_data else np.nan
    )
    
    # Pre-filter data by country code for faster access later
    # This creates separate GeoDataFrames for each country
    ee_nuts3 = baltic_nuts3[baltic_nuts3['CNTR_CODE'] == 'EE'].copy()  # Estonia
    lt_nuts3 = baltic_nuts3[baltic_nuts3['CNTR_CODE'] == 'LT'].copy()  # Lithuania
    
    # Store in a dictionary for easy access by country code
    country_nuts3 = {
        'EE': ee_nuts3,
        'LT': lt_nuts3,
        'all': baltic_nuts3  # Keep the complete dataset too
    }
    
    return country_nuts3, selected_nuts3

# Load geographic data once at startup (major performance optimization)
country_nuts3, selected_nuts3 = load_nuts3_data()

# Create dictionaries to cache visualizations
# This prevents rebuilding the same visualizations repeatedly
map_cache = {}              # Cache for maps by region_id
time_series_cache = {}      # Cache for time series charts
speedometer_cache = {}      # Cache for gauge/speedometer visualizations
dashboard_cache = {}        # Cache for complete dashboards by region_id

Loading NUTS-3 GeoJSON data...


In [8]:
def create_nuts3_map(region_id):
    """
    Create an interactive map showing NUTS-3 regions with appropriate coloring
    based on shelter usage percentages. Uses country-specific focus depending
    on which region is selected.
    
    Parameters:
    region_id (str): NUTS-3 region identifier (e.g., "EE001")
    
    Returns:
    panel.pane.HTML: Panel pane containing the interactive Folium map
    """
    # Check if this map is already in cache to avoid rebuilding
    if region_id in map_cache:
        return map_cache[region_id]
    
    # Define appropriate map centers and zoom levels for each country
    # These coordinates position the maps to properly show each country
    country_settings = {
        "EE": {"center": [59.0, 25.0], "zoom": 7},  # Estonia - centered on country
        "LT": {"center": [55.0, 24.0], "zoom": 7}   # Lithuania - centered on country
    }
    
    # Get country code from the first two characters of region_id (e.g., "EE" from "EE001")
    country_code = region_id[:2]
    
    # Set map settings based on selected country (or use default if country not found)
    map_settings = country_settings.get(country_code, {"center": [55.5, 25.0], "zoom": 6})
    
    # Create a Folium map with country-specific center and zoom
    m = folium.Map(
        location=map_settings["center"], 
        zoom_start=map_settings["zoom"], 
        tiles='CartoDB positron'  # Light gray basemap for better visibility of data
    )
    
    # Create an interactive tooltip that appears when hovering over regions
    # Shows detailed shelter information for each region
    tooltip = GeoJsonTooltip(
        # Fields from the GeoDataFrame to display
        fields=['NUTS_ID', 'NAME_LATN', 'Official_Shelter_Latest', 'Suggested_Shelter_Latest', 
                'Population_protected', 'Shelters_Used_Pct'],
        # Labels for each field in the tooltip
        aliases=['NUTS ID:', 'Region Name:', 'Official number of shelters:', 'Potential shelters:', 
                'Population covered (%):', 'Shelters Used (%):'],
        localize=True,     # Format numbers appropriately
        sticky=True,       # Tooltip stays visible when mouse is over it
        labels=True,       # Show field labels
        # Custom CSS styling for tooltip appearance
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """
    )
    
    # Helper function to determine region color based on shelter usage percentage
    # This creates the red-yellow-green coloring scheme
    def get_color_by_percentage(shelters_used_pct):
        if shelters_used_pct < 30:
            return "lightcoral"  # Red for low values (0-30%)
        elif shelters_used_pct < 70:
            return "khaki"       # Yellow for medium values (30-70%)
        else:
            return "lightgreen"  # Green for high values (70-100%)
    
    # Add all regions from the country as a background layer
    # These are shown in light gray
    folium.GeoJson(
        country_nuts3[country_code],  # All regions from the selected country
        name='Country NUTS 3 Regions',
        style_function=lambda feature: {
            'fillColor': 'lightgray',
            'weight': 0.5,
            'color': 'gray',
            'fillOpacity': 0.3,  # Semi-transparent
        },
    ).add_to(m)
    
    # Filter to just the regions with shelter data for this country
    country_selected = selected_nuts3[selected_nuts3['CNTR_CODE'] == country_code]
    
    # Add highlighted regions with shelter data on top
    # These are colored based on their shelter usage percentage
    folium.GeoJson(
        country_selected,
        name='Selected NUTS 3 Regions',
        style_function=lambda feature: {
            # Color based on shelter usage percentage
            'fillColor': get_color_by_percentage(feature['properties']['Shelters_Used_Pct']),
            # Thicker border for the selected region
            'weight': 2 if feature['properties']['NUTS_ID'] == region_id else 1,
            'color': 'black',
            # Higher opacity for the selected region
            'fillOpacity': 0.8 if feature['properties']['NUTS_ID'] == region_id else 0.7,
        },
        tooltip=tooltip,  # Add the interactive tooltip
        # Highlight effect when hovering
        highlight_function=lambda x: {'weight': 3, 'color': 'black'},
    ).add_to(m)
    
    # Convert Folium map to HTML and wrap in a Panel pane for dashboard integration
    # Set a fixed height but still responsive width to prevent scrollbars
    map_pane = pn.pane.HTML(m._repr_html_(), height=400, width=600)
    
    # Cache the map for future use
    map_cache[region_id] = map_pane
    
    return map_pane


In [9]:
def create_speedometer(value, title="Completion Percentage"):
    """
    Create a speedometer/gauge visualization using Plotly with caching
    The gauge bar color changes based on the value to match map coloring
    
    Parameters:
    value (float): Value to display on the gauge (0-100)
    title (str): Title for the gauge
    
    Returns:
    panel.pane.Plotly: Panel pane containing the Plotly gauge
    """
    # Create a cache key
    cache_key = f"{title}_{value}"
    
    # Check if speedometer is already in cache
    if cache_key in speedometer_cache:
        return speedometer_cache[cache_key]
    
    # Determine bar color based on value - matching map colors
    if value < 30:
        bar_color = "lightcoral"  # Red for low values (0-30%)
    elif value < 70:
        bar_color = "khaki"       # Yellow for medium values (30-70%)
    else:
        bar_color = "lightgreen"  # Green for high values (70-100%)
    
    fig = go.Figure(go.Indicator(
        mode="gauge+number",
        value=value,
        title={'text': title, 'font': {'size': 16}},
        number={'font': {'size': 20, 'color': 'darkblue'}},
        gauge={
            'axis': {
                'range': [0, 100], 
                'tickwidth': 1, 
                'tickcolor': "darkblue",
                'tickvals': [0, 25, 50, 75, 100],  # Explicit tick values
                'ticktext': ['0', '25', '50', '75', '100'],  # Explicit tick labels
                'tickfont': {'size': 12}  # Larger font for tick values
            },
            'bar': {'color': bar_color},  # Bar color matches map coloring scheme
            'bgcolor': "white",  # Setting background to white, removing the colored steps
            'threshold': {
                'line': {'color': "black", 'width': 2},
                'thickness': 0.75,
                'value': value
            }
        }
    ))
    
    fig.update_layout(
        height=200,
        width=260,
        margin=dict(l=40, r=40, t=50, b=30)  # Increased left and right margins to show full scale values
    )
    
    # Create panel pane
    speedometer_pane = pn.pane.Plotly(fig)
    
    # Cache the speedometer
    speedometer_cache[cache_key] = speedometer_pane
    
    return speedometer_pane

In [10]:
def create_time_series(region_id, metric_type, metric_name, color):
    """
    Create a time series chart showing the trend of a specific shelter metric.
    Uses Bokeh to create an interactive line chart with data points.
    
    Parameters:
    region_id (str): NUTS-3 region identifier (e.g., "EE001")
    metric_type (str): Type of metric to display ('official_shelters', 'suggested_shelters', 'population_protected')
    metric_name (str): Display name for the metric to show as chart title
    color (str): Line color for the chart (e.g., "blue", "green", "purple")
    
    Returns:
    panel.pane.Bokeh: Panel pane containing the Bokeh time series chart
    """
    # Create a cache key from region and metric type
    cache_key = f"{region_id}_{metric_type}"
    
    # Check if chart is already in cache to avoid rebuilding
    if cache_key in time_series_cache:
        return time_series_cache[cache_key]
        
    # Get the data for this region
    region = region_data[region_id]
    
    # Set y values and label based on which metric type was requested
    if metric_type == 'official_shelters':
        y_values = region['official_shelters']  # Number of official shelters
        y_label = "Official Shelters"
    elif metric_type == 'suggested_shelters':
        y_values = region['suggested_shelters']  # Number of potential shelters
        y_label = "Suggested Shelters"
    elif metric_type == 'population_protected':
        y_values = region['population_protected']  # Population coverage percentage
        y_label = "Population Protected (%)"
    
    # Create a Bokeh ColumnDataSource with the data
    # This connects the data to the visualization
    source = ColumnDataSource(data=dict(
        year=region['years'],  # X-axis: years
        value=y_values         # Y-axis: metric values
    ))
    
    # Create the Bokeh figure/plot with appropriate settings
    p = figure(title=f"{metric_name}", 
               width=350, 
               height=180,
               toolbar_location=None)  # No toolbar for cleaner look
    
    # Add a line connecting the data points
    p.line('year', 'value', line_width=3, color=color, source=source)
    
    # Add scatter points at each data point
    p.scatter('year', 'value', color=color, size=8, alpha=0.7, source=source)
    
    # Style the chart for better appearance
    p.grid.grid_line_alpha = 0.3  # Lighter grid lines
    p.xaxis.axis_label = "Year"   # Keep X-axis label
    # Remove y-axis label for cleaner look
    # p.yaxis.axis_label = y_label  # Y-axis label removed
    
    # Further minimizing for cleaner appearance
    p.title.text_font_size = '11pt'
    p.title.text_font = 'helvetica'
    p.title.text_font_style = 'normal'  # Instead of italic (default in some Bokeh themes)
    p.title.align = 'center'
    p.title.text_color = 'black'  # Ensure high contrast
    
    # Reduce axis line width for more subtle appearance
    p.axis.axis_line_width = 1
    p.axis.minor_tick_line_width = 0  # Remove minor ticks
    p.axis.major_tick_line_width = 1  # Thinner major ticks
    
    # Reduce the number of grid lines on both axes
    from bokeh.models import FixedTicker
    import math
    
    # For x-axis, use only ~5 years (assuming years are like 2020, 2021, etc.)
    years = region['years']
    if len(years) > 5:
        # Calculate step size to get ~5 ticks
        step = max(1, len(years) // 5)
        selected_years = years[::step]
        p.xaxis.ticker = FixedTicker(ticks=selected_years)
    
    # For y-axis, calculate appropriate ticks based on data range
    if metric_type == 'population_protected':
        # For percentage chart, use 0, 25, 50, 75, 100
        p.yaxis.ticker = FixedTicker(ticks=[0, 25, 50, 75, 100])
    else:
        # For all other charts, use round increments with balanced number of ticks
        y_min = min(y_values)
        y_max = max(y_values)
        
        # Target 4-6 tick marks
        target_ticks = 5
        
        # Calculate the ideal increment to get approximately the target number of ticks
        ideal_increment = (y_max - y_min) / (target_ticks - 1)
        
        # Find the nearest "nice" increment (1, 2, 5, 10, 20, 50, etc.)
        magnitude = 10 ** math.floor(math.log10(ideal_increment))
        
        if ideal_increment / magnitude < 1.5:
            nice_increment = magnitude  # Use 1x the magnitude (1, 10, 100, etc.)
        elif ideal_increment / magnitude < 3.5:
            nice_increment = 2 * magnitude  # Use 2x the magnitude (2, 20, 200, etc.)
        elif ideal_increment / magnitude < 7.5:
            nice_increment = 5 * magnitude  # Use 5x the magnitude (5, 50, 500, etc.)
        else:
            nice_increment = 10 * magnitude  # Use 10x the magnitude (10, 100, 1000, etc.)
        
        # Ensure the increment is at least 1
        nice_increment = max(1, nice_increment)
        
        # Create nice round-numbered ticks
        # Round down the minimum to the nearest nice_increment
        nice_min = math.floor(y_min / nice_increment) * nice_increment
        # Round up the maximum to the nearest nice_increment
        nice_max = math.ceil(y_max / nice_increment) * nice_increment
        
        # Generate ticks
        ticks = [nice_min + i * nice_increment for i in range(int((nice_max - nice_min) / nice_increment) + 1)]
        
        p.yaxis.ticker = FixedTicker(ticks=ticks)
    
    # Set y-axis range to 0-100 for the population_protected chart (percentage)
    if metric_type == 'population_protected':
        p.y_range.start = 0
        p.y_range.end = 100
    
    # Create Panel pane to contain the Bokeh chart
    chart_pane = pn.pane.Bokeh(p)
    
    # Cache the chart for future use
    time_series_cache[cache_key] = chart_pane
    
    return chart_pane

In [11]:
def create_download_button(region_id):
    """
    Create a download button to allow users to download region-specific data.
    This provides files with detailed shelter data for each region.
    
    Parameters:
    region_id (str or tuple): NUTS-3 region identifier (e.g., "EE001")
                             Can be tuple when coming from RadioButtonGroup
    
    Returns:
    panel.Column: Panel column containing the download button and descriptive text
    """
    # Extract the first element if region_id is a tuple (from RadioButtonGroup)
    if isinstance(region_id, tuple):
        region_id = region_id[0]
    
    # Get the region data and name
    region = region_data[region_id]
    region_name = region['name']
    
    # Map region IDs to local file paths
    # Here we use a single common file but could be different for each region
    files = {
        "EE001": "../data/possible_shelters.geojson",
        "LT011": "../data/possible_shelters.geojson"
    }
    
    # Get the appropriate file path for this region
    file_path = files.get(region_id)
    
    # Create a download button widget for the file
    download_button = pn.widgets.FileDownload(
        file=file_path,
        button_type="primary",  # Blue button
        label=f"Download {region_name} Data",  # Button text
        icon="download",        # Download icon
        width=250,
        align="center"
    )
    
    # Create a container with descriptive text and the download button
    # Reduced vertical spacing for more compact layout
    download_container = pn.Column(
        pn.pane.Markdown("### Download Region Data", align='center', margin=(2, 10, 2, 10)),
        pn.pane.Markdown(f"Download potential shelter data for {region_name}:", align='center', margin=(2, 10, 2, 10)),
        download_button,
        width=350,
        align='center',
        margin=5  # Reduced margin between elements
    )
    
    return download_container

In [12]:
def create_dashboard(region_id):
    """
    Create the main dashboard layout for a specific region.
    Assembles all visualization components (map, charts, gauges)
    into a cohesive layout.
    
    Parameters:
    region_id (str): NUTS-3 region identifier (e.g., "EE001")
    
    Returns:
    panel.Column: Complete dashboard layout as a Panel column
    """
    # Check if dashboard is already in cache to avoid rebuilding
    if region_id in dashboard_cache:
        return dashboard_cache[region_id]
        
    # Get the region data
    region = region_data[region_id]
    
    # Create dashboard title with region name
    title = pn.pane.Markdown(f"# {region['name']} Dashboard", align='center')
    
    # Create the map for this region
    region_map = create_nuts3_map(region_id)
    
    # Create three time series charts with different colors for each metric
    time_series1 = create_time_series(region_id, 'official_shelters', "Official shelters (n)", "blue")
    time_series2 = create_time_series(region_id, 'suggested_shelters', "Suggested shelters (n)", "green")
    time_series3 = create_time_series(region_id, 'population_protected', "People protected by shelters (%)", "purple")
    
    # Create two speedometers with the KPI values calculated earlier
    kpi_data = region['kpi_data']
    speedometer1 = create_speedometer(kpi_data['shelters_used'], "Shelters Used %")
    speedometer2 = create_speedometer(kpi_data['population_coverage'], "Population Covered %")
    
    # Create download button for region-specific data
    download_button = create_download_button(region_id)
    
    # Create the map section with negative margin to reduce space below it
    map_section = pn.Column(
        pn.Spacer(height=12),  # Add space above the map to push it down
        region_map,
        align='center',
        width=600,
        margin=(0, 0, -20, 0)  # Top, right, bottom, left - negative bottom margin to pull content up
    )

    # Create descriptive text for each speedometer
    speedometer_descriptions = [
        "**Percent of official shelters out of all generated shelters**",
        "**Percent of the population covered by official shelters**"
    ]
    
    # Create speedometer section with two gauges side by side
    speedometer_section = pn.Row(
        # Left speedometer with description
        pn.Column(
            speedometer1,
            pn.pane.Markdown(speedometer_descriptions[0], align='center', 
                            styles={'font-size': '14px', 'line-height': '1.1', 'margin-top': '-20px', 'margin-bottom': '0px', 'width': '100%', 'text-align': 'center'}),
            width=290,
            align='center',
            margin=0  # No margin to reduce spacing
        ),
        # Right speedometer with description
        pn.Column(
            speedometer2,
            pn.pane.Markdown(speedometer_descriptions[1], align='center', 
                            styles={'font-size': '14px', 'line-height': '1.1', 'margin-top': '-20px', 'margin-bottom': '0px', 'width': '100%', 'text-align': 'center'}),
            width=290,
            align='center',
            margin=0  # No margin to reduce spacing
        ),
        align='center',
        width=600,
        margin=(0, 0, 0, 0)  # Explicit zero margin
    )
    
    # Create color chart legend for the map
    color_chart = pn.pane.HTML("""
        <p align="center" style="font-size:14px">Regions colored by percentage of shelters used</p>
        <div align="center" style="font-size:12px">
            <span style="background-color:lightcoral; padding:0 10px; margin:0 5px; border:1px solid black;">0-30%</span>
            <span style="background-color:khaki; padding:0 10px; margin:0 5px; border:1px solid black;">30-70%</span>
            <span style="background-color:lightgreen; padding:0 10px; margin:0 5px; border:1px solid black;">70-100%</span>
        </div>
    """, width=600, align='center')
    
    # Create the time series section with all three charts stacked
    time_series_section = pn.Column(
        pn.pane.Markdown("## Shelter Data Metrics (2020-2024)", align='center'),
        pn.Column(
            time_series1,
            pn.Spacer(height=10),  # Small spacer between charts
            time_series2,
            pn.Spacer(height=10),  # Small spacer between charts
            time_series3,
            align='center',
            width=350
        ),
        align='center',
        width=400
    )
    
    # Create the full dashboard layout
    # Two columns side by side - left for map/gauges, right for charts/download
    dashboard = pn.Column(
        title,  # Dashboard title
        pn.Spacer(height=20),  # Space after title
        pn.Row(
            # Left side - Map with speedometers and color chart below it
            pn.Column(
                map_section, 
                # No spacer between map and speedometers
                speedometer_section,
                pn.Spacer(height=10),  # Small space between speedometers and color chart
                color_chart,
                width=600
            ),
            
            # Right side - Time series charts and download button
            pn.Column(
                time_series_section,
                pn.Spacer(height=10),  # Reduced spacer height
                download_button,
                width=400
            ),
            align='start'  # Align at the top of the row
        )
    )
    
    # Cache the dashboard for future use
    dashboard_cache[region_id] = dashboard
    
    return dashboard

In [13]:
# Create a region selector widget
region_options = [(region_id, region_data[region_id]['name']) for region_id in region_data]
region_selector = pn.widgets.RadioButtonGroup(
    options=region_options,
    button_type='primary',
    value='EE001'  # Default value
)

# Define a function to get the dashboard for the selected region
@pn.depends(region_selector.param.value)
def get_dashboard(region_id):
    """
    Get the dashboard for the selected region
    
    Parameters:
    region_id (str or tuple): NUTS-3 region identifier
    
    Returns:
    panel.Column: Dashboard for the selected region
    """
    if region_id is None:
        region_id = 'EE001'  # Default region
    # Check if region_id is a tuple (from RadioButtonGroup) and extract the first element
    if isinstance(region_id, tuple):
        region_id = region_id[0]
    return create_dashboard(region_id)

# Create the app layout
app_title = pn.pane.Markdown("# Find Me Shelter: Baltic NUTS 3 Regions Dashboard", align='center')
intro_text = pn.pane.Markdown("Select a region to view shelter data:")

# Assemble the final layout
main_layout = pn.Column(
    app_title,
    pn.Row(
        intro_text,
        pn.Spacer(width=20),
        region_selector
    ),
    pn.panel(get_dashboard),
    sizing_mode='stretch_width'  # Make the layout adjust to available width
)



In [14]:
# Display the dashboard
main_layout

Column(sizing_mode='stretch_width')
    [0] Markdown(str, align='center')
    [1] Row
        [0] Markdown(str)
        [1] Spacer(width=20)
        [2] RadioButtonGroup(button_type='primary', options=[('EE001', 'Põhja-Eesti'),...])
    [2] ParamFunction(function, _pane=Column, defer_load=False)